In [0]:
CREATE WIDGET TEXT catalog DEFAULT "";

In [0]:
USE CATALOG identifier(:catalog);

-- DROP SCHEMA IF EXISTS moviebuster_gold CASCADE;
-- CREATE SCHEMA moviebuster_gold;

USE SCHEMA moviebuster_gold;

In [0]:
CREATE OR REPLACE TABLE dim_customer(
  customer_id INT NOT NULL COMMENT 'Unique identifier for each customer',
  first_name STRING COMMENT 'Customer first name',
  last_name STRING COMMENT 'Customer last name',
  email STRING COMMENT 'Customer email address',
  active BOOLEAN COMMENT 'Indicates whether the customer is currently active',
  create_date DATE COMMENT 'Date when the customer record was created',
  address STRING COMMENT 'Street address of the customer',
  postal_code STRING COMMENT 'Postal code associated with the customer address',
  city STRING COMMENT 'City of the customer address',
  country STRING COMMENT 'Country of the customer address',
  CONSTRAINT pk_customer PRIMARY KEY (customer_id)
);

ALTER TABLE dim_customer
SET TAGS (
    'classification' = 'internal',
    'data_owner' = 'owner@databricks.com',
    'refresh_frequency' = 'daily'
);

ALTER TABLE dim_customer
ALTER COLUMN first_name
SET TAGS ('class.name');

ALTER TABLE dim_customer
ALTER COLUMN last_name
SET TAGS ('class.name');

ALTER TABLE dim_customer
ALTER COLUMN email
SET TAGS ('class.email_address');

ALTER TABLE dim_customer
ALTER COLUMN address
SET TAGS ('class.location');

ALTER TABLE dim_customer
ALTER COLUMN postal_code
SET TAGS ('class.location');

ALTER TABLE dim_customer
ALTER COLUMN city
SET TAGS ('class.location');

ALTER TABLE dim_customer
ALTER COLUMN country
SET TAGS ('class.location');

INSERT INTO dim_customer
  SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    c.active,
    c.create_date,
    a.address,
    a.postal_code,
    ci.city,
    co.country
  FROM
    identifier(:catalog || '.moviebuster_bronze.customer') c
      JOIN identifier(:catalog || '.moviebuster_bronze.address') a
        ON c.address_id = a.address_id
      JOIN identifier(:catalog || '.moviebuster_bronze.city') ci
        ON a.city_id = ci.city_id
      JOIN identifier(:catalog || '.moviebuster_bronze.country') co
        ON ci.country_id = co.country_id;

In [0]:
CREATE OR REPLACE TABLE dim_staff(
  staff_id INT NOT NULL COMMENT 'Unique identifier for each staff member',
  first_name STRING COMMENT 'Staff member first name',
  last_name STRING COMMENT 'Staff member last name',
  email STRING COMMENT 'Staff member email address',
  active BOOLEAN COMMENT 'Indicates whether the staff member is currently active',
  username STRING COMMENT 'Login username for the staff member',
  address STRING COMMENT 'Street address of the staff member',
  postal_code STRING COMMENT 'Postal code associated with the staff address',
  city STRING COMMENT 'City of the staff address',
  country STRING COMMENT 'Country of the staff address',
  staff_store_id INT COMMENT 'Identifier of the store where the staff member works',
  CONSTRAINT pk_staff PRIMARY KEY (staff_id)
);

ALTER TABLE dim_staff
SET TAGS (
    'classification' = 'internal',
    'data_owner' = 'owner@databricks.com',
    'refresh_frequency' = 'daily'
);

ALTER TABLE dim_staff
ALTER COLUMN first_name
SET TAGS ('class.name');

ALTER TABLE dim_staff
ALTER COLUMN last_name
SET TAGS ('class.name');

ALTER TABLE dim_staff
ALTER COLUMN email
SET TAGS ('class.email_address');

ALTER TABLE dim_staff
ALTER COLUMN address
SET TAGS ('class.location');

ALTER TABLE dim_staff
ALTER COLUMN postal_code
SET TAGS ('class.location');

ALTER TABLE dim_staff
ALTER COLUMN city
SET TAGS ('class.location');

ALTER TABLE dim_staff
ALTER COLUMN country
SET TAGS ('class.location');

INSERT INTO dim_staff
  SELECT
    s.staff_id,
    s.first_name,
    s.last_name,
    s.email,
    s.active,
    s.username,
    a.address,
    a.postal_code,
    ci.city,
    co.country,
    st.store_id AS staff_store_id
  FROM
    identifier(:catalog || '.moviebuster_bronze.staff') s
      JOIN identifier(:catalog || '.moviebuster_bronze.address') a
        ON s.address_id = a.address_id
      JOIN identifier(:catalog || '.moviebuster_bronze.city') ci
        ON a.city_id = ci.city_id
      JOIN identifier(:catalog || '.moviebuster_bronze.country') co
        ON ci.country_id = co.country_id
      JOIN identifier(:catalog || '.moviebuster_bronze.store') st
        ON s.store_id = st.store_id;

In [0]:
CREATE OR REPLACE TABLE dim_film(
  film_id INT NOT NULL COMMENT 'Unique identifier for each film',
  title STRING COMMENT 'Title of the film',
  release_year INT COMMENT 'Year the film was released',
  rating STRING COMMENT 'Film rating (e.g. G, PG, R)',
  rental_duration INT COMMENT 'Number of days the film can be rented',
  rental_rate DECIMAL(5, 2) COMMENT 'Rental price of the film',
  replacement_cost DECIMAL(5, 2) COMMENT 'Cost to replace the film if lost or damaged',
  special_features STRING COMMENT 'Comma-separated list of special features (e.g. Deleted Scenes, Behind the Scenes)',
  language STRING COMMENT 'Primary language of the film',
  categories STRING COMMENT 'Comma-separated list of categories/genres the film belongs to',
  CONSTRAINT pk_film PRIMARY KEY (film_id)
);

ALTER TABLE dim_film
SET TAGS (
    'classification' = 'internal',
    'data_owner' = 'owner@databricks.com',
    'refresh_frequency' = 'daily'
);

INSERT INTO dim_film
  SELECT
    f.film_id,
    f.title,
    f.release_year,
    f.rating,
    f.rental_duration,
    f.rental_rate,
    f.replacement_cost,
    f.special_features,
    l.name AS language,
    STRING_AGG(DISTINCT cat.name) AS categories
  FROM
    identifier(:catalog || '.moviebuster_bronze.film') f
      JOIN identifier(:catalog || '.moviebuster_bronze.language') l
        ON f.language_id = l.language_id
      LEFT JOIN identifier(:catalog || '.moviebuster_bronze.film_category') fc
        ON f.film_id = fc.film_id
      LEFT JOIN identifier(:catalog || '.moviebuster_bronze.category') cat
        ON fc.category_id = cat.category_id
  GROUP BY
    f.film_id,
    f.title,
    f.release_year,
    f.rating,
    f.rental_duration,
    f.rental_rate,
    f.replacement_cost,
    f.special_features,
    l.name;

In [0]:
CREATE OR REPLACE TABLE dim_store(
  store_id INT NOT NULL COMMENT 'Unique identifier for each store',
  manager_staff_id INT COMMENT 'Identifier of the staff member managing the store',
  address STRING COMMENT 'Street address of the store',
  postal_code STRING COMMENT 'Postal code associated with the store location',
  city STRING COMMENT 'City where the store is located',
  country STRING COMMENT 'Country where the store is located',
  CONSTRAINT pk_store PRIMARY KEY (store_id)
);

ALTER TABLE dim_store
SET TAGS (
    'classification' = 'internal',
    'data_owner' = 'owner@databricks.com',
    'refresh_frequency' = 'daily'
);

ALTER TABLE dim_store
ALTER COLUMN address
SET TAGS ('class.location');

ALTER TABLE dim_store
ALTER COLUMN postal_code
SET TAGS ('class.location');

ALTER TABLE dim_store
ALTER COLUMN city
SET TAGS ('class.location');

ALTER TABLE dim_store
ALTER COLUMN country
SET TAGS ('class.location');

INSERT INTO dim_store
  SELECT
    st.store_id,
    st.manager_staff_id,
    a.address,
    a.postal_code,
    ci.city,
    co.country
  FROM
    identifier(:catalog || '.moviebuster_bronze.store') st
      JOIN identifier(:catalog || '.moviebuster_bronze.address') a
        ON st.address_id = a.address_id
      JOIN identifier(:catalog || '.moviebuster_bronze.city') ci
        ON a.city_id = ci.city_id
      JOIN identifier(:catalog || '.moviebuster_bronze.country') co
        ON ci.country_id = co.country_id;

In [0]:
CREATE OR REPLACE TABLE rental_fact(
  rental_id INT NOT NULL COMMENT 'Unique identifier for each rental transaction',
  rental_date DATE COMMENT 'Date when the rental occurred',
  return_date DATE COMMENT 'Date when the rental was returned',
  rental_duration INT COMMENT 'Number of days the rental lasted (calculated as return_date - rental_date)',
  amount DECIMAL(10, 2) COMMENT 'Total amount paid for the rental',
  customer_id INT COMMENT 'Reference to the customer who made the rental',
  staff_id INT COMMENT 'Reference to the staff member who handled the rental',
  film_id INT COMMENT 'Reference to the film that was rented',
  store_id INT COMMENT 'Reference to the store where the rental was made',
  CONSTRAINT pk_rental_fact PRIMARY KEY (rental_id),
  CONSTRAINT fk_rental_customer FOREIGN KEY (customer_id) REFERENCES dim_customer (customer_id),
  CONSTRAINT fk_rental_staff FOREIGN KEY (staff_id) REFERENCES dim_staff (staff_id),
  CONSTRAINT fk_rental_film FOREIGN KEY (film_id) REFERENCES dim_film (film_id),
  CONSTRAINT fk_rental_store FOREIGN KEY (store_id) REFERENCES dim_store (store_id)
);

INSERT INTO rental_fact
  SELECT
    r.rental_id,
    r.rental_date,
    r.return_date,
    DATEDIFF(r.return_date, r.rental_date) AS rental_duration,
    p.amount,
    r.customer_id,
    r.staff_id,
    i.film_id,
    i.store_id
  FROM
    identifier(:catalog || '.moviebuster_bronze.rental') r
      JOIN identifier(:catalog || '.moviebuster_bronze.payment') p
        ON r.rental_id = p.rental_id
      JOIN identifier(:catalog || '.moviebuster_bronze.inventory') i
        ON r.inventory_id = i.inventory_id;

In [0]:
CREATE OR REPLACE VIEW rental_fact_view AS
SELECT
  -- Fact
  f.rental_id,
  f.rental_date,
  f.return_date,
  f.rental_duration,
  f.amount,
  -- Customer Dimension
  dc.customer_id,
  dc.first_name AS customer_first_name,
  dc.last_name AS customer_last_name,
  dc.email AS customer_email,
  dc.active AS customer_active,
  dc.address AS customer_address,
  dc.postal_code AS customer_postal_code,
  dc.city AS customer_city,
  dc.country AS customer_country,
  -- Staff Dimension
  ds.staff_id,
  ds.first_name AS staff_first_name,
  ds.last_name AS staff_last_name,
  ds.email AS staff_email,
  ds.username AS staff_username,
  ds.active AS staff_active,
  ds.address AS staff_address,
  ds.postal_code AS staff_postal_code,
  ds.city AS staff_city,
  ds.country AS staff_country,
  ds.staff_store_id,
  -- Film Dimension
  df.film_id,
  df.title AS film_title,
  df.release_year AS film_release_year,
  df.rating as film_rating,
  df.rental_duration AS film_rental_duration,
  df.rental_rate AS film_rental_rate,
  df.replacement_cost AS film_replacement_cost,
  df.special_features AS film_special_features,
  df.language AS film_language,
  df.categories AS film_categories,
  -- Store Dimension
  dstore.store_id,
  dstore.manager_staff_id,
  dstore.address AS store_address,
  dstore.postal_code AS store_postal_code,
  dstore.city AS store_city,
  dstore.country AS store_country
FROM
  rental_fact f
    JOIN dim_customer dc
      ON f.customer_id = dc.customer_id
    JOIN dim_staff ds
      ON f.staff_id = ds.staff_id
    JOIN dim_film df
      ON f.film_id = df.film_id
    JOIN dim_store dstore
      ON f.store_id = dstore.store_id
WHERE
  YEAR(f.rental_date) = '2005';

## Metrics

In [0]:
CREATE
OR REPLACE VIEW rental_metrics (
  `Rental Date`,
  `Return Date`,
  `Customer ID`,
  `Customer City`,
  `Customer Country`,
  `Staff ID`,
  `Staff City`,
  `Staff Country`,
  `Film ID`,
  `Film Title`,
  `Film Category`,
  `Film Release Year`,
  `Film Rating`,
  `Store ID`,
  `Store City`,
  `Store Country`,
  `Total Rentals`,
  `Total Amount`,
  `Average Rental Duration`,
  `Average Rental Amount`,
  `Maximum Rental Amount`,
  `Minimum Rental Amount`,
  `Distinct Customers`,
  `Distinct Films Rented`,
  `Rentals Per Customer`,
  `Rentals Per Staff`,
  `Rentals Per Store`,
  `Return Rate`,
  `Daily Rental Revenue`
) WITH METRICS LANGUAGE YAML
COMMENT 'Metric view for analyzing rentals, revenue, and customer behavior across stores and staff.' AS $$
version: 0.1
source: marat_levit.moviebuster_gold.rental_fact_view
dimensions:
  - name: Rental Date
    expr: rental_date
  - name: Return Date
    expr: return_date
  - name: Customer ID
    expr: customer_id
  - name: Customer City
    expr: customer_city
  - name: Customer Country
    expr: customer_country
  - name: Staff ID
    expr: staff_id
  - name: Staff City
    expr: staff_city
  - name: Staff Country
    expr: staff_country
  - name: Film ID
    expr: film_id
  - name: Film Title
    expr: film_title
  - name: Film Category
    expr: film_categories
  - name: Film Release Year
    expr: film_release_year
  - name: Film Rating
    expr: film_rating
  - name: Store ID
    expr: store_id
  - name: Store City
    expr: store_city
  - name: Store Country
    expr: store_country
measures:
  - name: Total Rentals
    expr: count(rental_id)
  - name: Total Amount
    expr: sum(amount)
  - name: Average Rental Duration
    expr: avg(rental_duration)
  - name: Average Rental Amount
    expr: avg(amount)
  - name: Maximum Rental Amount
    expr: max(amount)
  - name: Minimum Rental Amount
    expr: min(amount)
  - name: Distinct Customers
    expr: count(distinct customer_id)
  - name: Distinct Films Rented
    expr: count(distinct film_id)
  - name: Rentals Per Customer
    expr: count(rental_id) / count(distinct customer_id)
  - name: Rentals Per Staff
    expr: count(rental_id) / count(distinct staff_id)
  - name: Rentals Per Store
    expr: count(rental_id) / count(distinct store_id)
  - name: Return Rate
    expr: count(return_date) / count(rental_id)
  - name: Daily Rental Revenue
    expr: sum(amount) / datediff(max(rental_date), min(rental_date) + 1)
$$;

## ABAC - CLM

In [0]:
CREATE OR REPLACE FUNCTION mask_email(email STRING)
RETURNS STRING
RETURN regexp_replace(email, '(.*)@', '');

In [0]:
CREATE OR REPLACE POLICY mask_emails
ON SCHEMA moviebuster_gold
COLUMN MASK mask_email
TO `account users`
FOR TABLES
MATCH COLUMNS hasTag('class.email_address') AS email
ON COLUMN email;

## ABAC - RLS

In [0]:
-- CREATE OR REPLACE FUNCTION australia_filter(country STRING)
-- RETURNS STRING
-- RETURN country = 'Australia';

In [0]:
-- CREATE OR REPLACE POLICY australian_customers_only
-- ON SCHEMA moviebuster_gold
-- ROW FILTER australia_filter
-- TO `account users`
-- FOR TABLES
-- MATCH COLUMNS hasTag('class.location') AS country
-- USING COLUMNS (country);